## Florida Midterm Election Results by County — US Senate (2022)

By: Shirsho Dasgupta (2022)

The code scrapes live election results for the US Senate race for each Florida county.

The column names in the dataframe are generated only after the results have been scraped, meaning that the codes are reusable for different races and the dataframe dynamically sizes itself according to the number of candidates or issues on the ballot. It does not matter how many candidates there are or in which order they appear. 

The code then computes the vote shares of each candidate, generates the names of the candidates who are leading (or have won) and are runners-up, their party affiliations and their lead/win margin in each county. It also computes the leader/winner and their vote-share and lead/win margin for the entire state. 

The resulting dataframe for each race is then joined with previously prepared spreadsheets containing results from the prior election cycle (2016 for the Senate race). 

The code was set up to run at intervals of 10 minutes and write the final dataframe into a Google Sheet.

A sample final data frame can be found [here](https://github.com/shirshod/florida_midterms_2022/blob/main/county_results/senate_state/sen_state_scraper_report_11-13-2022-2009.csv).

A data dictionary can be found [here](https://github.com/shirshod/florida_midterms_2022/blob/main/county_results/county_dictionary.pdf). 

The final dataframe powered the Miami Herald’s live election maps:
1. [Live Updating: Florida Senate Election Results by County](https://www.datawrapper.de/_/u2slr/)
2. [Live Results: How Demings and Rubio are doing in Florida statewide](https://www.datawrapper.de/_/v2ISI/)

#### Sources:

[Florida Elections Watch Live Results — United States Senator](https://floridaelectionwatch.gov/ContestResultsByCounty/120000)

[Florida 2016 General Election Results — Federal Offices](https://results.elections.myflorida.com/Index.asp?ElectionDate=11/8/2016&DATAMODE=)

##### Note: 
Websites showing live results generally switch over to the next election and may not be displaying the data for the elections being discussed here. Please use [Wayback Machine](https://archive.org/web/) to view archived versions of the dataframe. 

### Importing libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import datetime as dt
import os
import re

In [2]:
### ignores copy warning
pd.options.mode.chained_assignment = None 

### Scraping county-level results

In [3]:
### creates directory to store final dataframe after exporting
os.makedirs("senate_state/", exist_ok = True)

### sets URL
url = "https://floridaelectionwatch.gov/ContestResultsByCounty/120000"

### retrieves data
page = requests.get(url)
### converts data to text for reading
soup = BeautifulSoup(page.text)

### stores time of processing/update
tnow = dt.datetime.now()
dt_string = tnow.strftime("%m-%d-%Y %H:%M")
update = "Last updated at " + str(dt_string)


## COMPUTING CANDIDATE LISTS

### stores HTML tags with candidate information
can_tags = soup.find_all("th", {"class": "text-right"})

### creates empty lists to write into
can_names = []
can_party = []
candidates = []

### loop runs through all the candidate tags
for i in range(0, len(can_tags)):

    ### modifying name and party-affiliation
    name_initial = can_tags[i].text.strip()
    name = " ".join(name_initial.split())
    party_initial = name_initial[-4:].replace(")", "")
    
    ### checks for write-in candidates
    if party_initial == "WRI":
        party = "WRITE-IN/OTHER"
    else:
        party = party_initial

    ### stores names of candidates and party-affiliation
    can_names.append(name)
    can_party.append(party)
    
    ### stores candidate details in the form "<name>, <party>"
    candidates.append(name)
    candidates.append(party)

### number of candidates
can_length = len(can_names)


## COMPUTING COUNTY LISTS

### stores tags with county names
county_tags = soup.find_all("td", style = "white-space:nowrap;")

### creates lists to store counties and counties with candidate names and parties
counties = []
county_names = []

### loop runs through county tags
for i in range(0, len(county_tags)):
    
    ### extracts and stores county names
    county = county_tags[i].text.strip()
    counties.append(county)
    
    ### new row initialized with timestamp for updation
    row = [update]
    
    ### adds county to row
    row.append(county)
    
    ### loop runs through all the candidates
    for j in range(0, len(candidates)):
        ### stores candidate names
        row.append(candidates[j])
    
    ### constructs list in the form "<county name> <candidate> <party>"     
    county_names.append(row)

    
## COMPUTING VOTES PER COUNTY 

### stores tags with tallied votes per candidate
row_tags = soup.find_all("tr")

### creates list to store tallied votes
can_votes = []

### loop runs through all the vote tags
for i in range(1, len(row_tags)-1):
    
    ### stores all tags containing individual votes
    nos = row_tags[i].find_all("td", {"class": "text-right"})
    
    ### creates list to store votes
    vote = []
    
    ### loop runs through list of all votes
    for j in range(0, len(nos)):
        
        ### extracts and stores individual votes
        number = nos[j].text.strip().replace(",", "")
        vote.append(number)
    
    ### stores list of votes per county
    can_votes.append(vote)


## COMPUTING VOTES BY COUNTY

### loop runs through all the counties
for i in range(0, len(county_names)):
    
    ### loop runs through all the candidates
    for j in range(0, can_length):
        ### extracts and stores votes
        county_names[i].append(int(can_votes[i][j]))

### converts list to dataframe      
df = pd.DataFrame(county_names)

### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Last updated at 11-13-2022 20:09,Alachua,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,WRITE-IN/OTHER,39190,55359,439,349,285,0,0,0,0
1,Last updated at 11-13-2022 20:09,Baker,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,WRITE-IN/OTHER,9431,1181,43,31,6,0,0,0,0
2,Last updated at 11-13-2022 20:09,Bay,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,WRITE-IN/OTHER,51587,14512,300,254,193,0,0,0,0
3,Last updated at 11-13-2022 20:09,Bradford,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,WRITE-IN/OTHER,8156,1941,37,55,19,0,0,0,0
4,Last updated at 11-13-2022 20:09,Brevard,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,WRITE-IN/OTHER,165109,98839,1233,1074,629,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Last updated at 11-13-2022 20:09,Union,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,WRITE-IN/OTHER,3916,580,13,22,7,0,0,0,0
63,Last updated at 11-13-2022 20:09,Volusia,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,WRITE-IN/OTHER,138717,84345,939,868,417,0,0,0,0
64,Last updated at 11-13-2022 20:09,Wakulla,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,WRITE-IN/OTHER,10480,4337,69,65,35,0,0,0,0
65,Last updated at 11-13-2022 20:09,Walton,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,WRITE-IN/OTHER,28178,6317,124,98,65,0,0,0,0


### Computing county-level total tallied votes

In [4]:
### computes number of columns
col_no = len(df.columns)

### creates new column to write total into
df[col_no] = ""

### creates list to store tallied votes
votes_tallied = []

### loop runs through dataframe
for i in range(0, len(df)):
    
    ### initializes variable to store total
    total = 0
    
    ### loop runs through the votes
    for j in range(can_length*2+2, col_no):
        ### adds votes to total
        total = total + df[j][i]
    
    ### adds total to list
    votes_tallied.append(total)
    ### writes total into new column
    df[col_no][i] = total

### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,Last updated at 11-13-2022 20:09,Alachua,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,39190,55359,439,349,285,0,0,0,0,95622
1,Last updated at 11-13-2022 20:09,Baker,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,9431,1181,43,31,6,0,0,0,0,10692
2,Last updated at 11-13-2022 20:09,Bay,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,51587,14512,300,254,193,0,0,0,0,66846
3,Last updated at 11-13-2022 20:09,Bradford,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,8156,1941,37,55,19,0,0,0,0,10208
4,Last updated at 11-13-2022 20:09,Brevard,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,165109,98839,1233,1074,629,0,0,0,0,266884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Last updated at 11-13-2022 20:09,Union,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,3916,580,13,22,7,0,0,0,0,4538
63,Last updated at 11-13-2022 20:09,Volusia,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,138717,84345,939,868,417,0,0,0,0,225286
64,Last updated at 11-13-2022 20:09,Wakulla,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,10480,4337,69,65,35,0,0,0,0,14986
65,Last updated at 11-13-2022 20:09,Walton,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,28178,6317,124,98,65,0,0,0,0,34782


### Computing county-level vote shares, leads and margins

In [5]:
### number of columns in new dataframe
col_no_2 = len(df.columns)

### creates new list to store index numbers of newly created columns
new_cols = []

### loop runs through number of candidates and four more for other details
for i in range(0, (can_length + 4)):   
    ### creates (n+4) new columns for "n" candidates
    df[col_no + i + 1] = " "
    ### stores numbers in a list as strings
    new_cols.append(str(col_no + i + 1))
    
### loop runs through each row of dataframe
for i in range(0, len(df)):
    
    ### creates list to store votes obtained
    votes_obtained = []
    
    ### stores the total votes tallied for each row
    total = votes_tallied[i]
    
    ### if votes have been tallied
    if total != 0: 
        
        ## TALLYING VOTES

        ### loop runs through columns storing votes
        for j in range(can_length*2+2, col_no_2-1):

            ### stores votes as a list
            votes_obtained.append(int(df[j][i]))

        ## COMPUTING VOTE-SHARES

        ### loop runs through all the candidates
        for k in range(0, can_length):
            ### column-index of a candidate is given by corresponding element in "new_cols"
            ### first candidate (k = 0) will be stored in column at index-0 in the "new_cols" list and so on
            column_no = int(new_cols[k])
            ### computes percent and writes into the column
            df[column_no][i] = (votes_obtained[k]/total*100).round(2)
            
        ## COMPUTING LEADING CANDIDATES AND WIN MARGINS

        ### computes and stores the highest vote-tally for the "votes_obtained" list
        max_value = max(votes_obtained)

        ### checks if there are more elements with same value (in case of ties)
        if votes_obtained.count(max_value) > 1:

            ### method returns indices of all duplicate highest vote-tallies
            indices = []
            def find_indices(list_to_check, item_to_find):
                return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

            ### stores indices of all duplicate highest vote-tallies
            indices = find_indices(votes_obtained, max_value)

            ### constructs string of all candidate names corresponding to highest vote-tallies
            string = []
            for ind in range(0, len(indices)):
                index = indices[ind]
                string.append(can_names[index])

            ### stores value as "TIED"
            lead_party = "TIED"
            ### stores candidate names with tied votes
            lead_can = " and ".join([", ".join(string[:-1]),string[-1]] if len(string) > 2 else string)
            ### stores margin of difference as 0
            margin = 0
            margin_pp = 0

        else:
            ### computes and stores the index at which the highest vote is stored in the "votes_obtained" list
            max_index = votes_obtained.index(max_value)
            ### stores name of party corresponding to highest vote-tally in the fourth-last column
            ### if highest vote-tally is second (index=1) element in "votes_obtained"
            ### the second element (index=1) in "party" list is the corresponding datapoint
            lead_party = can_party[max_index]
            ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
            lead_can = can_names[max_index]

            ### computes and stores the second highest vote-tally for the "votes_obtained" list
            second_max_value = max(votes_obtained, key = lambda x: min(votes_obtained)-1 if (x == max_value) else x)
            ### computes and stores the raw number of votes between first and second placed candidates
            margin = max_value - second_max_value
            ### computes the difference in percent points between first and second placed candidates
            margin_pp = ((max_value/total*100)-(second_max_value/total*100)).round(2)

        ### stores index-number of the fourth-last element of "new_cols" list (this will be the fourth-last column)
        lead_col = int(new_cols[-4])
        ### stores name of party corresponding to highest vote-tally in the fourth-last column
        ### if highest vote-tally is second (index=1) element in "votes_obtained"
        ### the second element (index=1) in "party" list is the corresponding datapoint
        df[lead_col][i] = lead_party
        ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
        df[lead_col+1][i] = lead_can

        ### stores index-number of the penultimate element of "new_cols" list (this will be the penultimate column)
        raw_margin_col = int(new_cols[-2])
        ### stores raw margin of difference in penultimate column
        df[raw_margin_col][i] = margin

        ### stores index-number of the last element of "new_cols" list (this will be the last column)
        margin_pp_col = int(new_cols[-1])
        ### stores percent points difference in last column
        df[margin_pp_col][i] = margin_pp
                  
    ### if no votes have been tallied        
    else:
        ### loop runs through all candidates
        for n in range(0, can_length):
            ### column-index of a candidate is given by corresponding element in "new_cols"
            ### first candidate (k = 0) will be stored in column at index-0 in the "new_cols" list and so on
            column_no = int(new_cols[n])
            ### stores vote-share as 0 and writes into the column
            df[column_no][i] = 0
        
        ### sets values 
        lead_party = "No votes tallied"
        lead_can = "No votes tallied"
        margin = 0
        margin_pp = 0
        
        ### stores index-number of the fourth-last element of "new_cols" list (this will be the fourth-last column)
        lead_col = int(new_cols[-4])
        ### stores name of party corresponding to highest vote-tally in the fourth-last column
        ### if highest vote-tally is second (index=1) element in "votes_obtained"
        ### the second element (index=1) in "party" list is the corresponding datapoint
        df[lead_col][i] = lead_party
        ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
        df[lead_col+1][i] = lead_can

        ### stores index-number of the penultimate element of "new_cols" list (this will be the penultimate column)
        raw_margin_col = int(new_cols[-2])
        ### stores raw margin of difference in penultimate column
        df[raw_margin_col][i] = margin

        ### stores index-number of the last element of "new_cols" list (this will be the last column)
        margin_pp_col = int(new_cols[-1])
        ### stores percent points difference in last column
        df[margin_pp_col][i] = margin_pp       
        
### replaces null values with "0"
df.fillna(0, inplace = True)    
   
### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,Last updated at 11-13-2022 20:09,Alachua,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.36,0.30,0.0,0.0,0.0,0.0,DEM,Val Demings (DEM),16169,16.91
1,Last updated at 11-13-2022 20:09,Baker,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.29,0.06,0.0,0.0,0.0,0.0,REP,Marco Rubio (REP),8250,77.16
2,Last updated at 11-13-2022 20:09,Bay,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.38,0.29,0.0,0.0,0.0,0.0,REP,Marco Rubio (REP),37075,55.46
3,Last updated at 11-13-2022 20:09,Bradford,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.54,0.19,0.0,0.0,0.0,0.0,REP,Marco Rubio (REP),6215,60.88
4,Last updated at 11-13-2022 20:09,Brevard,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.40,0.24,0.0,0.0,0.0,0.0,REP,Marco Rubio (REP),66270,24.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Last updated at 11-13-2022 20:09,Union,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.48,0.15,0.0,0.0,0.0,0.0,REP,Marco Rubio (REP),3336,73.51
63,Last updated at 11-13-2022 20:09,Volusia,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.39,0.19,0.0,0.0,0.0,0.0,REP,Marco Rubio (REP),54372,24.13
64,Last updated at 11-13-2022 20:09,Wakulla,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.43,0.23,0.0,0.0,0.0,0.0,REP,Marco Rubio (REP),6143,40.99
65,Last updated at 11-13-2022 20:09,Walton,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.28,0.19,0.0,0.0,0.0,0.0,REP,Marco Rubio (REP),21861,62.85


### Computing overall state vote shares, leads and margins

In [6]:
## COMPUTING COUNTY-LEVEL TOTALS

### storing number of columns
new_col_no = len(df.columns)

### stores column number for last column
total_col = can_length*3 + 2

### stores and writes total votes tallied in the state
ct_total = df[total_col].sum()
df[new_col_no] = ct_total

### creates new list to store index numbers of newly created columns
ct_cols = []

### stores new length of dataframe
ct_new_col_no = len(df.columns)

### if votes have been tallied
if ct_total != 0:

    ## COMPUTING COUNTY-LEVEL VOTE SHARES

    ### computes sum of all votes per candidate
    ct_can_votes = []
    for j in range(can_length*2+2, col_no):
        ct_can_votes.append(df[j].sum())

    ### computes vote-shares of candidates
    for i in range(0, can_length):
        df[ct_new_col_no + i] = (ct_can_votes[i]/ct_total*100).round(2)

    ## COMPUTING LEADING CANDIDATES AND WIN MARGINS

    ### computes new length of dataframe
    final_col_no = len(df.columns)

    ### stores overall maximum votes
    ct_max_value = max(ct_can_votes)

    ### if no votes have been tallied
    #if ct_max_value == 0:

        #ct_lead_party = "N/A"
        #ct_lead_can = "No votes tallied."
        #ct_margin = 0
        #ct_margin_pp = 0

    ### if there are more than one instances of highest votes (in case of a tie)
    if ct_can_votes.count(ct_max_value) > 1:

        ### method returns indices of all duplicate highest vote-tallies
        indices = []
        def find_indices(list_to_check, item_to_find):
            return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

        ### stores indices of all duplicate highest vote-tallies
        indices = find_indices(ct_can_votes, ct_max_value)

        ### constructs string of all candidate names corresponding to highest votes tallied
        string = []
        for ind in range(0, len(indices)):
            index = indices[ind]
            string.append(can_names[index])

        ### stores leading party as tied
        ct_lead_party = "TIED"
        ### stores names of tied candidates
        ct_lead_can = " and ".join([", ".join(string[:-1]),string[-1]] if len(string) > 2 else string)

        ### stores margins as "0"
        ct_margin = 0
        ct_margin_pp = 0

        ### stores vote share for candidate with highest vote-tally
        ct_lead_voteshare = (ct_max_value/ct_total*100).round(2)
    
    ### if there is a clear winner/leader
    else: 

        ### computes index of the highest tallied vote
        ct_max_index = ct_can_votes.index(ct_max_value)

        ### stores name of corresponding party and candidates
        ct_lead_party = can_party[ct_max_index]
        ct_lead_can = can_names[ct_max_index]

        ### computes second highest tallied vote
        second_ct_max_value = max(ct_can_votes, key = lambda x: min(ct_can_votes)-1 if (x == ct_max_value) else x)
        ### computes index of the second-highest tallied vote
        second_ct_max_index = ct_can_votes.index(second_ct_max_value)
        
        ### stores name of corresponding party and candidates
        second_ct_party = can_party[second_ct_max_index]
        second_ct_can = can_names[second_ct_max_index]

        ### stores margin of difference and the margin as percent points
        ct_margin = ct_max_value - second_ct_max_value
        ct_margin_pp = ((ct_max_value/ct_total*100)-(second_ct_max_value/ct_total*100)).round(2)

        ### stores the vote share of the highest tallied vote
        ct_lead_voteshare = (ct_max_value/ct_total*100).round(2)
        ### stores the vote share of the second highest tallied vote
        second_ct_voteshare = (second_ct_max_value/ct_total*100).round(2)

    ### writes into cells
    df[final_col_no + 1] = ct_lead_party
    df[final_col_no + 2] = ct_lead_can
    df[final_col_no + 3] = ct_margin
    df[final_col_no + 4] = ct_margin_pp
    df[final_col_no + 5] = ct_lead_voteshare
    df[final_col_no + 6] = second_ct_party
    df[final_col_no + 7] = second_ct_can
    df[final_col_no + 8] = second_ct_voteshare

### if no votes have been tallied
else: 
    
    ### sets vote-shares of candidates as 0
    for i in range(0, can_length):
        df[ct_new_col_no + i] = 0
        
    ### computes new length of dataframe
    final_col_no = len(df.columns)
    
    ### sets values
    ct_lead_party = "No votes tallied"
    ct_lead_can = "No votes tallied"
    ct_margin = 0
    ct_margin_pp = 0
    ct_lead_voteshare = 0
    second_ct_party = "No votes tallied"
    second_ct_can = "No votes tallied"
    second_ct_voteshare = 0
    
    ### writes into cells
    df[final_col_no + 1] = ct_lead_party
    df[final_col_no + 2] = ct_lead_can
    df[final_col_no + 3] = ct_margin
    df[final_col_no + 4] = ct_margin_pp
    df[final_col_no + 5] = ct_lead_voteshare
    df[final_col_no + 6] = second_ct_party
    df[final_col_no + 7] = second_ct_can
    df[final_col_no + 8] = second_ct_voteshare

### replaces null values with "0"
df.fillna(0, inplace = True)    
   
### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,51,52,54,55,56,57,58,59,60,61
0,Last updated at 11-13-2022 20:09,Alachua,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
1,Last updated at 11-13-2022 20:09,Baker,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
2,Last updated at 11-13-2022 20:09,Bay,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
3,Last updated at 11-13-2022 20:09,Bradford,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
4,Last updated at 11-13-2022 20:09,Brevard,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Last updated at 11-13-2022 20:09,Union,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
63,Last updated at 11-13-2022 20:09,Volusia,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
64,Last updated at 11-13-2022 20:09,Wakulla,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
65,Last updated at 11-13-2022 20:09,Walton,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26


### Computing and inserting column names

In [7]:
### creates list to store column names with two initial elements
cols = ["LAST_UPDATE_2022", "COUNTY"]

### loop runs through all the candidates
for i in range(0, can_length):
    ### generates column names for each candidate and adds to list
    cols.append("CANDIDATE_NAME_" + str(i+1))
    cols.append("CANDIDATE_PARTY_" + str(i+1))
    
for i in range(0, can_length):
    cols.append("VOTES_WON_" + str(i+1))

### adds column to list
cols.append("TOTAL_VOTES_TALLIED")

### loop runs through all the candidates
for i in range(0, can_length):
    ### generates column name for each candidate and adds to list
    cols.append("VOTE_SHARE_" + str(i+1))

### adds columns to list
cols.append("LEAD_PARTY")
cols.append("LEAD_CANDIDATE")
cols.append("RAW_MARGIN")
cols.append("MARGIN_PERCENTPOINTS")
cols.append("ST_TOTAL_VOTES_TALLIED")

### loop runs through all the candidates
for i in range(0, can_length):
    ### generates column names for each candidate and adds to list
    cols.append("ST_VOTE_SHARE_" + str(i+1))

### adding column names for statewide results
cols.append("ST_LEAD_PARTY")
cols.append("ST_LEAD_CANDIDATE")
cols.append("ST_RAW_MARGIN")
cols.append("ST_MARGIN_PERCENTPOINTS")
cols.append("ST_LEAD_VOTESHARE")
cols.append("ST_SECOND_PARTY")
cols.append("ST_SECOND_CANDIDATE")
cols.append("ST_SECOND_VOTESHARE")

### replaces column indices with generated column names
df.columns = cols

### displays dataframe
df

,LAST_UPDATE_2022,COUNTY,CANDIDATE_NAME_1,CANDIDATE_PARTY_1,CANDIDATE_NAME_2,CANDIDATE_PARTY_2,CANDIDATE_NAME_3,CANDIDATE_PARTY_3,CANDIDATE_NAME_4,CANDIDATE_PARTY_4,...,ST_VOTE_SHARE_8,ST_VOTE_SHARE_9,ST_LEAD_PARTY,ST_LEAD_CANDIDATE,ST_RAW_MARGIN,ST_MARGIN_PERCENTPOINTS,ST_LEAD_VOTESHARE,ST_SECOND_PARTY,ST_SECOND_CANDIDATE,ST_SECOND_VOTESHARE
0,Last updated at 11-13-2022 20:09,Alachua,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
1,Last updated at 11-13-2022 20:09,Baker,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
2,Last updated at 11-13-2022 20:09,Bay,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
3,Last updated at 11-13-2022 20:09,Bradford,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
4,Last updated at 11-13-2022 20:09,Brevard,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Last updated at 11-13-2022 20:09,Union,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
63,Last updated at 11-13-2022 20:09,Volusia,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
64,Last updated at 11-13-2022 20:09,Wakulla,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26
65,Last updated at 11-13-2022 20:09,Walton,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,0.0,0.0,REP,Marco Rubio (REP),1272824,16.43,57.69,DEM,Val Demings (DEM),41.26


### Merging final dataframe with dataset of prior results

In [8]:
### importing dataset with demographics and prior results
df_prior = pd.read_csv("state_sen_prior.csv",  keep_default_na = False)

### merging 
df_master = pd.merge(df, df_prior, on = "COUNTY")

### stores time
dt_string = tnow.strftime("%m-%d-%Y-%H%M")
### constructs filename
report = "sen_state_scraper_report_" + str(dt_string) + ".csv"
path = "senate_state/" + report

### exports dataframe 
df_master.to_csv(path, index = False)

### displays dataframe
df_master

,LAST_UPDATE_2022,COUNTY,CANDIDATE_NAME_1,CANDIDATE_PARTY_1,CANDIDATE_NAME_2,CANDIDATE_PARTY_2,CANDIDATE_NAME_3,CANDIDATE_PARTY_3,CANDIDATE_NAME_4,CANDIDATE_PARTY_4,...,ST_SECOND_VOTESHARE,FIPS,WINNER_PARTY_16,WINNER_CANDIDATE_16,VOTE_SHARE_WINNER_16,MARGIN_PERCENTPOINTS_16,ST_WINNER_CANDIDATE_16,ST_WINNER_VOTESHARE_16,ST_MARGIN_PERCENTPOINTS_16,ST_WINNER_PARTY_16
0,Last updated at 11-13-2022 20:09,Alachua,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,41.26,12001,DEM,Patrick Murphy (DEM),54.5,11.9,Marco Rubio (REP),52,7.7,REP
1,Last updated at 11-13-2022 20:09,Baker,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,41.26,12003,REP,Marco Rubio (REP),79.5,62.4,Marco Rubio (REP),52,7.7,REP
2,Last updated at 11-13-2022 20:09,Bay,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,41.26,12005,REP,Marco Rubio (REP),72.5,49.5,Marco Rubio (REP),52,7.7,REP
3,Last updated at 11-13-2022 20:09,Bradford,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,41.26,12007,REP,Marco Rubio (REP),73.5,50.8,Marco Rubio (REP),52,7.7,REP
4,Last updated at 11-13-2022 20:09,Brevard,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,41.26,12009,REP,Marco Rubio (REP),58.3,22.0,Marco Rubio (REP),52,7.7,REP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Last updated at 11-13-2022 20:09,Union,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,41.26,12125,REP,Marco Rubio (REP),77.4,58.3,Marco Rubio (REP),52,7.7,REP
63,Last updated at 11-13-2022 20:09,Volusia,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,41.26,12127,REP,Marco Rubio (REP),54.5,13.8,Marco Rubio (REP),52,7.7,REP
64,Last updated at 11-13-2022 20:09,Wakulla,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,41.26,12129,REP,Marco Rubio (REP),64.3,32.4,Marco Rubio (REP),52,7.7,REP
65,Last updated at 11-13-2022 20:09,Walton,Marco Rubio (REP),REP,Val Demings (DEM),DEM,Dennis Misigoy (LPF),LPF,Steven B. Grant (NPA),NPA,...,41.26,12131,REP,Marco Rubio (REP),77.2,58.2,Marco Rubio (REP),52,7.7,REP
